In [1]:
import ast
import random

import pandas as pd

df = pd.read_csv('../output_data/detailed_travel_routes.csv')

df

,id,Domain,Goal,Plan,Detailed Plan
0,0,"[[-121.401001, 38.667599], [-121.2897144693877...","McClellan Airfield (SACRAMENTO, CA) to Hotel S...","[['Drive west.', 'Turn right after 582 meters ...","[[{'instruction': 'Drive west.', 'name': '-', ..."
1,1,"[[-86.44039045, 37.76657014], [-85.64796181454...","BRECKINRIDGE MEMORIAL HOSPITAL (HARDINSBURG, K...","[['Drive south.', 'Turn right after 11 meters ...","[[{'instruction': 'Drive south.', 'name': '-',..."
2,2,"[[-90.07257594727255, 29.95639325], [-90.07819...","Saenger Theatre (New Orleans, LA) to Charity H...","[['Drive southwest on North Rampart Street.', ...",[[{'instruction': 'Drive southwest on North Ra...
3,3,"[[-74.00448291111701, 40.7135665], [-73.496747...","Surrogate's Court (New York, NY) to ST JOHN'S ...","[['Drive northeast on Elk Street.', 'Turn left...",[[{'instruction': 'Drive northeast on Elk Stre...
4,4,"[[-105.1517817, 39.95190922], [-104.8943343, 3...",CENTURA HEALTH-AVISTA ADVENTIST HOSPITAL (LOUI...,"[['Drive southeast.', 'Turn left onto Health P...","[[{'instruction': 'Drive southeast.', 'name': ..."
...,...,...,...,...,...
495,495,"[[-75.598662, 38.370412], [-76.05799123, 38.54...","Gillis-Grier House (Salisbury, MD) to ADVENTIS...","[['Drive southwest on North Division Street.',...",[[{'instruction': 'Drive southwest on North Di...
496,496,"[[-81.04407659335502, 34.00904355], [-81.04072...","Lace House (Columbia, SC) to Building at 1210-...","[['Drive north on Gadsden Street.', 'Turn righ...",[[{'instruction': 'Drive north on Gadsden Stre...
497,497,"[[-78.87293133680211, 42.8848777], [-78.497961...","Hotel Lafayette (Buffalo, NY) to US Post Offic...","[['Drive east on Clinton Street.', 'Turn left ...",[[{'instruction': 'Drive east on Clinton Stree...
498,498,"[[-83.63421550067734, 32.842396353042204], [-8...","Cowles House (Macon, GA) to Emerson-Holmes Bui...","[['Drive southeast on Bond Street.', 'Turn lef...",[[{'instruction': 'Drive southeast on Bond Str...


In [2]:
df['Domain'] = df['Domain'].apply(lambda x: ast.literal_eval(x.replace('coordinates = ', '').strip()))

df['Plan'] = df['Plan'].apply(ast.literal_eval)

df['Detailed Plan'] = df['Detailed Plan'].apply(ast.literal_eval)

In [3]:
df['Plan'] = df['Plan'].apply(lambda p: p[0])

random.seed(42)

def get_question_and_ground_truth(row):

    plan = row['Plan']
    actions = ['Turn', 'Keep']

    # Choosing step
    steps_with_actions = [i for i, step in enumerate(row['Plan']) if any(action in step for action in actions)]

    if steps_with_actions:
        step_idx = random.choice(steps_with_actions)
    else:
       return None
    
    step = plan[step_idx]

    # Flipping directions
    if 'left' in step:
        direction = 'right'
    elif 'right' in step:
        direction = 'left'
    else:
        return None

    if 'Keep' in step:
        action = 'keeping'
    else:
        action = 'turning'

    return (f"Does {action} {direction} in Step {step_idx + 1} taint the preconditions for the rest of the directions?", "Yes")


df[['Question', 'Ground Truth']] = df.apply(get_question_and_ground_truth, axis=1, result_type="expand")

df = df.dropna()

df = df.sample(n=100, random_state=42)

df


,id,Domain,Goal,Plan,Detailed Plan,Question,Ground Truth
361,361,"[[-80.47779254616646, 35.66661947471722], [-80...","Hambley-Wallace House (Salisbury, NC) to DAVIS...","[Drive southwest on South Fulton Street., Bear...",[[{'instruction': 'Drive southwest on South Fu...,Does turning right in Step 11 taint the precon...,Yes
73,73,"[[-86.129219, 32.35771], [-86.74307505, 31.636...","CAVHCS EAST CAMPUS (MONTGOMERY, AL) to GEORGIA...","[Drive northwest., Turn left after 25 meters o...","[[{'instruction': 'Drive northwest.', 'name': ...",Does turning left in Step 3 taint the precondi...,Yes
374,374,"[[-97.31290555, 32.89687799], [-98.11187, 32.8...",PARKWAY SURGICAL AND CARDIOVASCULAR HOSPITAL (...,"[Drive northeast., Turn left onto North Tarran...","[[{'instruction': 'Drive northeast.', 'name': ...",Does turning right in Step 11 taint the precon...,Yes
155,155,"[[-83.04134305522764, 42.36825451847435], [-83...","Dunbar Hospital (Detroit, MI) to Durant-Dort C...","[Drive southwest on Frederick Street., Turn ri...",[[{'instruction': 'Drive southwest on Frederic...,Does turning left in Step 4 taint the precondi...,Yes
104,104,"[[-70.88723563, 42.56098687], [-71.14424530677...",HEALTHSOUTH NEW ENGLAND REHABILIATION AT BEVER...,"[Drive north., Turn right after 227 meters or ...","[[{'instruction': 'Drive north.', 'name': '-',...",Does turning left in Step 13 taint the precond...,Yes
...,...,...,...,...,...,...,...
347,347,"[[-70.18166139926271, 43.7980084], [-70.225032...","Camp Hammond (Yarmouth, ME) to Roak Block (Aub...","[Drive northeast on Torrey Court., Turn left o...",[[{'instruction': 'Drive northeast on Torrey C...,Does turning right in Step 4 taint the precond...,Yes
86,86,"[[-87.71135017, 41.52310929], [-87.78897925766...","FRANCISCAN ST JAMES HEALTH (OLYMPIA FIELDS, IL...","[Drive north., Turn right after 33 meters or 4...","[[{'instruction': 'Drive north.', 'name': '-',...",Does turning left in Step 4 taint the precondi...,Yes
75,75,"[[-80.46752472474998, 35.66921166372531], [-80...","Community Building (Salisbury, NC) to Arista C...",[Drive northeast on North Main Street/US 29/US...,[[{'instruction': 'Drive northeast on North Ma...,Does turning left in Step 6 taint the precondi...,Yes
438,438,"[[-74.44758128795094, 40.47672630107233], [-74...",National Musical String Company (New Brunswick...,"[Drive south on Georges Road., Turn left onto ...",[[{'instruction': 'Drive south on Georges Road...,Does turning right in Step 2 taint the precond...,Yes


In [4]:
df = df[[
    'id',
    'Goal',
    'Plan',
    'Question',
    'Ground Truth',
]]

df = df.sort_values(by=['id'])

df.to_csv('../output_data/data-flow_analyses/taint_analysis.csv', index=False)

df

,id,Goal,Plan,Question,Ground Truth
0,0,"McClellan Airfield (SACRAMENTO, CA) to Hotel S...","[Drive west., Turn right after 582 meters or 7...",Does turning right in Step 3 taint the precond...,Yes
2,2,"Saenger Theatre (New Orleans, LA) to Charity H...","[Drive southwest on North Rampart Street., Tur...",Does turning right in Step 7 taint the precond...,Yes
9,9,"Atlas Theatre (Cheyenne, WY) to VA MEDICAL CEN...","[Drive east on East 16th Street., Turn left on...",Does turning right in Step 7 taint the precond...,Yes
11,11,"Temple Beth Israel (Portland, OR) to St. James...","[Drive south., Turn left onto Northwest Flande...",Does turning right in Step 2 taint the precond...,Yes
15,15,"Boyden High School (Salisbury, NC) to Waldensi...","[Drive east on Taylor Street., Turn right onto...",Does turning left in Step 3 taint the precondi...,Yes
...,...,...,...,...,...
490,490,"Old Baptist Parsonage (Scotch Plains, NJ) to M...","[Drive southwest., Turn right onto Park Avenue...",Does keeping right in Step 9 taint the precond...,Yes
491,491,"Hopkins, Gov. Stephen, House (Providence, RI) ...","[Drive southwest on Hopkins Street., Turn righ...",Does keeping left in Step 12 taint the precond...,Yes
494,494,"Sanborn Seminary (Kingston, NH) to St. John's ...","[Drive south., Turn right onto Main Street., T...",Does turning right in Step 4 taint the precond...,Yes
495,495,"Gillis-Grier House (Salisbury, MD) to ADVENTIS...","[Drive southwest on North Division Street., Tu...",Does turning right in Step 8 taint the precond...,Yes
